## <center> Simple Particle Collider Data Filtration Program </center>
#### <center> By Luke Surber, SurberL9@uw.edu </center>
### Data from measurements in many experimental contexts is a combination of both the desired signal and background noise (including both systematic and random uncertainty), the signal and noise must be separated in order to correctly study the phenomena without bias. Particle Collider experiments are one such context that interest me, this notebook will serve as a simple filtration program that utilizes widgets to illustrate the process.
### Particle colliders have significant implications for the field of HEP, this task of separating noise from signal is absolutely critical to operating a collider (like the LHC for instance) to yield maximum scientific benefit. 
##### <center> (Last edit: 03/17/2021) </center>




In [1]:
#!pip install --upgrade jax jaxlib #Assuming JAX isn't already installed on the local machine being used by the user.
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
key = random.PRNGKey(0)
import random as rand
import math
from ipywidgets import interact
import matplotlib.pyplot as plt
plt.style.use('dark_background')

def toydata(signal_mean,noise_mean,signal_std,noise_std, background_events, cutoff):
    size = 100000
    mean = jnp.array([signal_mean,noise_mean]).block_until_ready()#block_until_ready helps speed up
    cov = jnp.array([[signal_std**2,0],[0,noise_std**2]]).block_until_ready()
    x1, x2 = random.multivariate_normal(key, mean, cov, (size,)).T.block_until_ready() 
    fig, axs = plt.subplots(1,3, figsize=(36,12))
    axs[0].set_title("Signal and Background Noise", fontweight="bold", size=20)
    axs[0].set_ylabel("Events [1/2.5GeV]", size=20)
    axs[0].set_xlabel("Energy [GeV]", size=20)
    axs[0].hist(x1, 100, facecolor='darkgreen')
    axs[0].hist(x2[:background_events], 100, facecolor='darkblue', alpha=0.5)
    axs[0].set_xlim(0,250)
    axs[1].set_title("Experimentally Measured Data", fontweight="bold", size=20)
    axs[1].set_xlabel("Energy [GeV]", size=20)
    axs[1].hist(jnp.concatenate((x1,x2[:background_events])), 100, facecolor='purple')
    axs[1].set_xlim(0,250)
    axs[2].set_title("Filtered Dataset", fontweight="bold", size=20)
    axs[2].set_xlabel("Energy [GeV]", size=20)
    axs[2].hist(jnp.concatenate((x1,x2[:background_events]))[jnp.concatenate((x1,x2[:background_events])) > cutoff], 100, facecolor='white')
    axs[2].set_xlim(0,250)

interact(toydata, background_events=(0,100000,1), signal_mean=(100,150,1), noise_mean=(0,120,1),signal_std=(0.1,50,0.5), noise_std=(0.1,50,0.5), cutoff=(0,250,1));

interactive(children=(IntSlider(value=125, description='signal_mean', max=150, min=100), IntSlider(value=60, d…

#### In the above plots we see first, two distributions that represent signal (in green) and background noise (in blue), in the second plot we see what instruments see (green+blue), lastly we see the result of our simple filtration method, which is essentially a discrete truncation of the data set; the sliders provide the opportunity to vary this truncation point or "cutoff", the other sliders allow for the user to alter the background noise and signal distributions to simulate the possible scenarios that might arise and how it can be filtrated depending on this "cutoff" parameter.

#### TRYME: One example of a dataset like this being relevant to HEP would be when the LHC was measuring the mass of the Higgs Boson which is about 125 GeV, try setting signal_mean to 125 GeV and the cutoff to the standard 50GeV, now play around with the other parameters to simulate how simple filtration on a dataset for the Higgs could happen.
##### NOTE: the background_events slider indicates the number of events measured that are caused by the background noise, it spans from 0 to one hundred thousand which is the number of signal events (arbitrarily chosen by myself, MUCH higher than what's measured by the LHC). Varying this slider is akin to varying how sensitive the instrumentation is to the background.

##### MINOR BUG THAT'S NOT REALLY A BUG: The slider label for background_events isn't displaying the label in it's entirety and this is something I would like to fix, although this is purely aesthetic and doesn't affect the theory.